In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('resources/data/2007-2018-grant-allocations.csv')
df.head()

,Year,State,Org Name,Program Name,Award Amount
0,2018,AK,Alaska,CDBG,2958723.0
1,2018,AK,Alaska,ESG,235176.0
2,2018,AK,Alaska,HOME,3008138.0
3,2018,AK,Alaska,HTF,3000000.0
4,2018,AK,"Anchorage, AK",CDBG,1726068.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131149 entries, 0 to 131148
Data columns (total 5 columns):
Year            131149 non-null int64
State           131149 non-null object
Org Name        78176 non-null object
Program Name    131149 non-null object
Award Amount    131149 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 5.0+ MB


In [4]:
df.nunique()

Year               16
State              56
Org Name         4359
Program Name        8
Award Amount    83499
dtype: int64

In [32]:
df['Program Name'].unique()

array(['CDBG', 'ESG', 'HOME', 'HTF', 'HOPWA', 'CoC',
       'Emergency Shelter Grants Program', 'NSP'], dtype=object)

## Assess

- Org Name is not an important category at this time and can be removed
- Year is currently int but is better as string
- Data is to be grouped by State, Program, and Year
- Column names are capitalizaed and have spaces
- There are 56 codes for states, the regions outside of the United States are to be removed
- 'Emergency Shelter Grants Program' is used instead of 'ESG' abbreviation for some entries

## Clean

In [33]:
df_clean = df.copy()

#### Org Name column not required

Remove Org Name column

In [34]:
df_clean.drop('Org Name', axis=1, inplace=True)
df_clean.head()

,Year,State,Program Name,Award Amount
0,2018,AK,CDBG,2958723.0
1,2018,AK,ESG,235176.0
2,2018,AK,HOME,3008138.0
3,2018,AK,HTF,3000000.0
4,2018,AK,CDBG,1726068.0


#### Column names are capitalized with spaces

Convert column names to 'year', 'state', 'program', 'amount'

In [35]:
cols = ['year', 'state', 'program', 'amount']
df_clean.columns = cols
df_clean.head()

,year,state,program,amount
0,2018,AK,CDBG,2958723.0
1,2018,AK,ESG,235176.0
2,2018,AK,HOME,3008138.0
3,2018,AK,HTF,3000000.0
4,2018,AK,CDBG,1726068.0


#### Year as `int`

Convert `Year` to `str` with `.astype(str)`

In [36]:
df_clean.year = df_clean.year.astype(str)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131149 entries, 0 to 131148
Data columns (total 4 columns):
year       131149 non-null object
state      131149 non-null object
program    131149 non-null object
amount     131149 non-null float64
dtypes: float64(1), object(3)
memory usage: 4.0+ MB


#### Dataset contains additional region codes

Import list of US states codes and include with DC using `.isin()`

In [10]:
df_clean.state.unique()

array(['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL',
       'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA',
       'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE',
       'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI',
       'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV',
       'WY'], dtype=object)

In [11]:
# Read in state info
states = pd.read_csv('resources/data/states.csv')
states.head()

,abbr,name
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [16]:
# Create list of state codes and add DC
state_codes = list(states.abbr)
state_codes.append('DC')
state_codes[-10:]

['TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'DC']

In [37]:
df_clean = df_clean[df_clean.state.isin(state_codes)]
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129291 entries, 0 to 131148
Data columns (total 4 columns):
year       129291 non-null object
state      129291 non-null object
program    129291 non-null object
amount     129291 non-null float64
dtypes: float64(1), object(3)
memory usage: 4.9+ MB


In [18]:
df_clean.nunique()

year          16
state         51
program        8
amount     82186
dtype: int64

#### Full version and abbreviation of Emergency Shelter Grants Program is used

Use `np.where` to replace 'Emergency Shelter Grants Program' with 'ESG' in `program` column

In [39]:
df_clean.program = np.where(df_clean.program == 'Emergency Shelter Grants Program', 'ESG', df_clean.program)

In [40]:
df_clean.program.nunique()

7

#### Group data by state, year, program

In [41]:
df_clean = df_clean.groupby(['state', 'year', 'program']).sum()
df_clean.reset_index(inplace=True)
df_clean.head()

,state,year,program,amount
0,AK,2003,CDBG,5796000.0
1,AK,2003,ESG,190000.0
2,AK,2003,HOME,4114921.0
3,AK,2003,HOPWA,731120.0
4,AK,2004,CDBG,5544637.0


In [42]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4104 entries, 0 to 4103
Data columns (total 4 columns):
state      4104 non-null object
year       4104 non-null object
program    4104 non-null object
amount     4104 non-null float64
dtypes: float64(1), object(3)
memory usage: 128.3+ KB
